# Lyrics Crawler



## Outline





In [81]:
import os
import gzip
import re
import http.cookiejar
import urllib.request
import urllib.parse
from time import strftime
import requests #HTTP for Humans 
from bs4 import BeautifulSoup
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
import datetime as dt

import logging
from logging.handlers import RotatingFileHandler

In [64]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
loghandler = logging.handlers.TimedRotatingFileHandler(filename='logs/lyrics_crawler.log', when="midnight")
loghandler.setLevel(logging.INFO)
fileformatter = logging.Formatter('%(asctime)s - %(filename)s [%(levelname)s] >>> %(message)s')
loghandler.setFormatter(fileformatter)
logger.addHandler(loghandler)

In [76]:
def check_folderexist(path='img/'):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: 
            logger.exception("message")
            if exc.errno != errno.EEXIST:
                raise    

In [5]:

header = {
'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-CN;q=0.2',
'Cache-Control':'max-age=0',
'Connection':'keep-alive',
'Cookie':'PHPSESSID=ap27jqvh1enqk8nsvp9e01qau2; __atuvc=38%7C35; __atuvs=59a662f167ac5a52007',
'DNT':'1',
'Host':'mojim.com',
'Referer':'https://mojim.com/twh100951.htm',
'Upgrade-Insecure-Requests':'1',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36', 
}
   

** 取得歌手網頁 ** 

In [77]:
url = 'https://mojim.com/twza1.htm'
sess = requests.Session()
r = sess.get(url, headers = header)
print(r.url, r.status_code, r.history)
data = r.content
soup = BeautifulSoup(data, "html.parser")

ul_data = soup.find("ul", attrs={'class':['s_listA']})
li_data = ul_data.find_all("li")
whoandhrefs = []
for i in li_data:
    a = i.find('a')
    title=a.text
    href = a['href']
    whoandhrefs.append((title,href))

whoandhrefs_df = pd.DataFrame(whoandhrefs, columns=['title','href'])
whoandhrefs_df.shape

https://mojim.com/twza1.htm 200 []


(80, 2)

In [92]:
def check_complete(folder):
    try:
        if 'complete.flag' in  os.listdir(folder):
            return True
        else:
            return False
    except:
        return False
    

def setcompleteflag(folder):
    path = '{folder}{file}'.format(folder=folder, file='complete.flag')
    file = open(path,'w') 
    file.write(str(dt.datetime.now()))
    file.close()
    

def crawler(folder, url):
    sess = requests.Session()
    r = sess.get(url, headers = header)
    print( folder, r.url, r.status_code, r.history)
    data = r.content
    soup = BeautifulSoup(data, "html.parser")
    
    dd_data = soup.find_all("dd", attrs={'class':['hb2','hb3']})
    all_tracks = []
    for idx , part in enumerate(dd_data):
        albuminfo = part.find_all('span', attrs={'class':'hc1'})[0]
        if albuminfo.text in ['暫存']:
            continue
        album_str = albuminfo.find('a').contents[0]
        yearinfo = part.find_all('span', attrs={'class':'hc2'})[0]
        if len(yearinfo.contents)>2:
            year_str= yearinfo.contents[2]
        trackinfo = part.find_all('span', attrs={'class':['hc3','hc4']})
        for tmp in trackinfo:
            tracks = tmp.find_all('a')
            for track in tracks: 
                name = track.contents[0]
                link = track['href']
                all_tracks.append((album_str, year_str, name, link))

    all_tracks = pd.DataFrame(all_tracks)
    
    for idx, row in all_tracks.iterrows():
        try:
            htm = row[3]
            filname='{y}_{n}'.format(y=row[1],n=row[2])
            filname = filname.strip()
            path = '{folder}{file}'.format(folder=folder, file=filname)
            url = 'https://mojim.com{u}'.format(u=htm)
            r = sess.get(url, headers = header)
            lyrics_data = r.content
            soup = BeautifulSoup(lyrics_data, "html.parser")

            dl_data = soup.find_all("dl", attrs={'class':'fsZx1'})
            for idx , part in enumerate(dl_data):
                dt_data = part.find_all('dt')
                dd_data = part.find_all('dd')[0]
                lyrics = dd_data.contents
                lyrics = [w if isinstance(w,str) else '\n' for w in lyrics]
                lyric_str= ''.join(lyrics)
                file = open(path,'w') 
                file.write(lyric_str) 
                file.close() 
        except Exception as e:
            logger.exception("message")
            logger.error("error, {a},{b}".format(a=idx,b=filname))
    

In [97]:
#sess = requests.Session()
for idx, row in whoandhrefs_df.iterrows():
    folder = 'text/lyrics/{who}/'.format(who=row['title'])
    url = 'https://mojim.com{h}'.format(h=row['href'])
    check_folderexist(folder)
    if not check_complete(folder):
        crawler(folder, url)
        setcompleteflag(folder)



text/六哲/ https://mojim.com/twh108782.htm 200 []
text/鄭中基/ https://mojim.com/twh100168.htm 200 []
text/齊秦/ https://mojim.com/twh100155.htm 200 []
text/鹿晗/ https://mojim.com/twh135932.htm 200 []
text/朴樹/ https://mojim.com/twh106229.htm 200 []
text/TANK/ https://mojim.com/twh104824.htm 200 []
text/張杰/ https://mojim.com/twh104500.htm 200 []
text/曹格/ https://mojim.com/twh102360.htm 200 []
text/陳柏宇/ https://mojim.com/twh105054.htm 200 []
text/麥浚龍/ https://mojim.com/twh102248.htm 200 []
text/陳百強/ https://mojim.com/twh100446.htm 200 []
text/側田/ https://mojim.com/twh104790.htm 200 []
text/陳曉東/ https://mojim.com/twh100119.htm 200 []
text/蘇永康/ https://mojim.com/twh100184.htm 200 []
text/吳宗憲/ https://mojim.com/twh100037.htm 200 []
text/宋岳庭/ https://mojim.com/twh104300.htm 200 []
text/陳昇/ https://mojim.com/twh100108.htm 200 []
text/品冠/ https://mojim.com/twh101065.htm 200 []
text/鄭伊健/ https://mojim.com/twh100169.htm 200 []
text/謝霆鋒/ https://mojim.com/twh100180.htm 200 []


***

*** END ***

*** 
*** 以下備份抓一個歌手的流程 ***

** 1. 取得歌曲與網址列表**

In [32]:
#周杰倫
#url = 'https://mojim.com/twh100951.htm'
#folder = 'text/jaychou/'
#陳奕迅
#url = 'https://mojim.com/twh100111.htm'
#folder ='text/easonchen/'
#莫文蔚
#url = 'https://mojim.com/twh100098.htm'
#folder ='text/karenmok/'
#八三夭
#url = 'https://mojim.com/twh105532.htm'
#folder ='text/831/'
#五月天
url = 'https://mojim.com/twh100012.htm'
folder ='text/mayday/'



sess = requests.Session()
r = sess.get(url, headers = header)
print(r.url, r.status_code, r.history)
data = r.content
soup = BeautifulSoup(data, "html.parser")

https://mojim.com/twh100012.htm 200 []


In [33]:
dd_data = soup.find_all("dd", attrs={'class':['hb2','hb3']})
all_tracks = []
for idx , part in enumerate(dd_data):
    albuminfo = part.find_all('span', attrs={'class':'hc1'})[0]
    if albuminfo.text in ['暫存']:
        continue
    album_str = albuminfo.find('a').contents[0]
    yearinfo = part.find_all('span', attrs={'class':'hc2'})[0]
    if len(yearinfo.contents)>2:
        year_str= yearinfo.contents[2]
    trackinfo = part.find_all('span', attrs={'class':['hc3','hc4']})
    for tmp in trackinfo:
        tracks = tmp.find_all('a')
        for track in tracks: 
            name = track.contents[0]
            link = track['href']
            all_tracks.append((album_str, year_str, name, link))
        
all_tracks = pd.DataFrame(all_tracks)

In [1]:
#all_tracks

** 2. 取得歌詞 **

In [35]:
for idx, row in all_tracks.iterrows():
    try:
        htm = row[3]
        filname='{y}_{n}'.format(y=row[1],n=row[2])
        filname = filname.strip()
        path = '{folder}{file}'.format(folder=folder, file=filname)
        url = 'https://mojim.com{u}'.format(u=htm)
        r = sess.get(url, headers = header)
        lyrics_data = r.content
        soup = BeautifulSoup(lyrics_data, "html.parser")

        dl_data = soup.find_all("dl", attrs={'class':'fsZx1'})
        for idx , part in enumerate(dl_data):
            dt_data = part.find_all('dt')
            dd_data = part.find_all('dd')[0]
            lyrics = dd_data.contents
            lyrics = [w if isinstance(w,str) else '\n' for w in lyrics]
            lyric_str= ''.join(lyrics)
            #all_lyrics.append(lyric_str)
            file = open(path,'w') 
            file.write(lyric_str) 
            file.close() 
    except Exception as e:
        print(e)
        print('error, ',idx,', ',filname)
        



[Errno 2] No such file or directory: 'text/mayday/2017-02_もしも出会わなければ / 如果我們不曾相遇'
error,  0 ,  2017-02_もしも出会わなければ / 如果我們不曾相遇
[Errno 2] No such file or directory: 'text/mayday/2017-02_成功間近 / 成名在望'
error,  0 ,  2017-02_成功間近 / 成名在望
[Errno 2] No such file or directory: 'text/mayday/2017-02_兄弟 / 兄弟'
error,  0 ,  2017-02_兄弟 / 兄弟
[Errno 2] No such file or directory: 'text/mayday/2017-02_人生有限会社 / 人生有限公司'
error,  0 ,  2017-02_人生有限会社 / 人生有限公司
[Errno 2] No such file or directory: 'text/mayday/2017-02_あれから僕ら / 後來的我們'
error,  0 ,  2017-02_あれから僕ら / 後來的我們
[Errno 2] No such file or directory: 'text/mayday/2017-02_頑固 / 頑固'
error,  0 ,  2017-02_頑固 / 頑固
[Errno 2] No such file or directory: 'text/mayday/2017-02_Party Animal / 派對動物'
error,  0 ,  2017-02_Party Animal / 派對動物
[Errno 2] No such file or directory: 'text/mayday/2017-02_最高の一日 / 最好的一天'
error,  0 ,  2017-02_最高の一日 / 最好的一天
[Errno 2] No such file or directory: 'text/mayday/2017-02_少年漂流記 /少年他的奇幻漂流'
error,  0 ,  2017-02_少年漂流記 /少年他的奇幻漂流
[Errno 2] No such 